In [1]:
import numpy as np
import pandas as pd
import time
import timeit
from numba import cuda, config
import math
import consts
import sys
import h5py

Define GPU Kernel functions:  
__1. Dummy Segmenter__  
__2. Quenching__  
__3. Drifting__  

In [2]:
'''A dummy segmenter which takes in track information and divides it by number of segments per track
to produce arrays of track segments in GPU device'''
@cuda.jit
def DummySegmenter(trkHeader, trkPos, trkdE,
                   segHeader, segPos, segdE):
    
    index = cuda.grid(1)    
    
    if (index < segHeader.shape[0]): 
        tID         = int(index/nseg_per_track)
        
        pos         = trkPos[tID]
        
        dx          = (pos['end_x'] - pos['start_x'])/nseg_per_track
        dy          = (pos['end_y'] - pos['start_y'])/nseg_per_track
        dz          = (pos['end_z'] - pos['start_z'])/nseg_per_track
        
        cuda.syncthreads()
        segPos[index]['start_x'] = pos['start_x'] + dx*(index%nseg_per_track)
        segPos[index]['end_x']   = pos['start_x'] + dx*((index%nseg_per_track)+1)
        segPos[index]['start_y'] = pos['start_y'] + dy*(index%nseg_per_track)
        segPos[index]['end_y']   = pos['start_y'] + dy*((index%nseg_per_track)+1)
        segPos[index]['start_z'] = pos['start_z'] + dz*(index%nseg_per_track)
        segPos[index]['end_z']   = pos['start_z'] + dz*((index%nseg_per_track)+1)
        
        segdE[index]       = trkdE[tID]/nseg_per_track
        segHeader[index]   = trkHeader[tID]

In [3]:
'''GPU Quenching Kernel function'''
@cuda.jit
def Quenching(segPos, segdE, ionElectrons):
    
    index  = cuda.grid(1)         
    if (index < segPos.shape[0]):   
        
        pos        = segPos[index]
        dx         = math.sqrt((pos['end_x'] - pos['start_x']) ** 2 + 
                    (pos['end_y'] - pos['start_y']) ** 2 +
                    (pos['end_z'] - pos['start_z']) ** 2)
        dedx        = abs(segdE[index])*consts.GeVToMeV/dx
        epsilon     = consts.recombBeta/(consts.lArDensity * consts.eField) * dedx
        recomb      = math.log(consts.recombAlpha + epsilon) / epsilon
        
        ionElectrons[index]['nElectrons']  = recomb * segdE[index] * consts.GeVToMeV * consts.MeVToElectrons

In [4]:
'''GPU Drift Simulation Kernel function'''
@cuda.jit
def Drifting(segPos, ionElectrons):
    index  = cuda.grid(1)    
    if (index < segPos.shape[0]): 
        z = (segPos[index]['end_z'] + segPos[index]['start_z'])/2.    
        drift_time  = abs(z - consts.tpcPlaneZ)/ consts.vdrift;
        lifetime    = math.exp(-drift_time/consts.msTous)
        ionElectrons[index]['nElectrons'] *= lifetime
        ionElectrons[index]['long_diff']   = math.sqrt(drift_time) * consts.longDiff
        ionElectrons[index]['trans_diff']  = math.sqrt(drift_time) * consts.transDiff

  
__Read Track Data from input hdf5 file 
and initialize output file and arrays__
  

In [5]:
inFile  = h5py.File('FakeTrackDataSet.h5', 'r')
outFile = h5py.File('TrackSegmentDataSet.h5', 'w')

In [6]:
headType  = ([('run',int), ('subrun', int), ('spill', int), ('interactionID', int), ('trackID', int), ('pdg', int)])
header    = np.zeros(inFile['RunID'].shape[0], dtype=headType)

In [7]:
header['run']           = np.array(inFile['RunID'])
header['subrun']        = np.array(inFile['SubrunID'])
header['spill']         = np.array(inFile['SpillID'])
header['interactionID'] = np.array(inFile['InteractionID'])
header['trackID']       = np.array(inFile['TrackID'])
header['pdg']           = np.array(inFile['PDG'])

In [8]:
posType  = ([('start_x', float), ('end_x', float), ('start_y', float), ('end_y', float), ('start_z', float), ('end_z', float)])
trkPos   = np.zeros(inFile['StartXPos'].shape[0], dtype=posType)
trkPos['start_x'] = np.array(inFile['StartXPos'])
trkPos['end_x']   = np.array(inFile['EndXPos'])
trkPos['start_y'] = np.array(inFile['StartYPos'])
trkPos['end_y']   = np.array(inFile['EndYPos'])
trkPos['start_z'] = np.array(inFile['StartZPos'])
trkPos['end_z']   = np.array(inFile['EndZPos'])

In [9]:
dE = np.array(inFile['EnergyDep'])

In [10]:
header_device = cuda.to_device(header)
trkPos_device = cuda.to_device(trkPos)
dE_device     = cuda.to_device(dE)

In [11]:
ntracks = trkPos.shape[0]
nseg_per_track = 500
nout = ntracks*nseg_per_track

In [12]:
seg_header_device = cuda.to_device(np.zeros(nout, dtype=headType))
seg_pos_device    = cuda.to_device(np.zeros(nout, dtype=posType))
seg_dE_device     = cuda.to_device(np.zeros(nout, dtype=float))

In [13]:
chargeType          = ([('long_diff', float), ('trans_diff', float), ('nElectrons', float)])
ionElectrons_device = cuda.to_device(np.zeros(nout, dtype=chargeType))

__Define GPU thread configuration and
call the kernel functions__

In [14]:
threads_per_block = 512
blocks_per_grid   = int((nout+threads_per_block -1)/threads_per_block)

In [27]:
%%timeit -n100
DummySegmenter[blocks_per_grid, threads_per_block](header_device, trkPos_device, dE_device,
                                                   seg_header_device, seg_pos_device, seg_dE_device)

350 µs ± 30.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
cuda.synchronize()

In [28]:
%%timeit -n100
Quenching[blocks_per_grid, threads_per_block](seg_pos_device, seg_dE_device, ionElectrons_device)

211 µs ± 25 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
cuda.synchronize()

In [29]:
%%timeit -n100
Drifting[blocks_per_grid, threads_per_block](seg_pos_device, ionElectrons_device)

197 µs ± 53.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
cuda.synchronize()

__Copy output from device to host machine__

In [21]:
segPos  = seg_pos_device.copy_to_host()
segHead = seg_header_device.copy_to_host()
ionE    = ionElectrons_device.copy_to_host()

__Make a pandas data frame to locally inspect the output__

In [22]:
df_final = pd.DataFrame()

In [23]:
df_final['RunID']                = segHead[:]['run']
df_final['SubrunID']             = segHead[:]['subrun']
df_final['SpillID']              = segHead[:]['spill']
df_final['InteractionID']        = segHead[:]['interactionID']
df_final['trackID']              = segHead[:]['trackID']
df_final['PDG']                  = segHead[:]['pdg']
df_final['x_start']              = segPos[:]['start_x']
df_final['x_end']                = segPos[:]['end_x']
df_final['y_start']              = segPos[:]['start_y']
df_final['y_end']                = segPos[:]['end_y']
df_final['z_start']              = segPos[:]['start_z']
df_final['z_end']                = segPos[:]['end_z']
df_final['nElectrons']           = ionE[:]['nElectrons']
df_final['Long Diff']            = ionE[:]['long_diff']
df_final['Tran Diff']            = ionE[:]['trans_diff']

In [24]:
df_final

,RunID,SubrunID,SpillID,InteractionID,trackID,PDG,x_start,x_end,y_start,y_end,z_start,z_end,nElectrons,Long Diff,Tran Diff
0,1,1,0,0,0,13,-26.467743,-26.422064,-127.677666,-127.639941,64.699921,64.745773,3967.322517,0.000232,0.000609
1,1,1,0,0,0,13,-26.422064,-26.376385,-127.639941,-127.602216,64.745773,64.791626,3967.204250,0.000232,0.000609
2,1,1,0,0,0,13,-26.376385,-26.330705,-127.602216,-127.564491,64.791626,64.837478,3967.085987,0.000232,0.000609
3,1,1,0,0,0,13,-26.330705,-26.285026,-127.564491,-127.526766,64.837478,64.883330,3966.967727,0.000232,0.000609
4,1,1,0,0,0,13,-26.285026,-26.239347,-127.526766,-127.489041,64.883330,64.929183,3966.849470,0.000232,0.000609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1,1,19,199,1999,13,52.815390,52.850188,-23.070872,-23.024503,-90.587365,-90.552857,3761.608273,0.000122,0.000320
999996,1,1,19,199,1999,13,52.850188,52.884986,-23.024503,-22.978135,-90.552857,-90.518349,3761.523882,0.000122,0.000320
999997,1,1,19,199,1999,13,52.884986,52.919784,-22.978135,-22.931766,-90.518349,-90.483841,3761.439492,0.000122,0.000321
999998,1,1,19,199,1999,13,52.919784,52.954582,-22.931766,-22.885398,-90.483841,-90.449333,3761.355105,0.000122,0.000321


__Store output arrays in the output hdf5 file__

In [25]:
outFile.create_dataset("RunID",                    data=segHead[:]['run'])
outFile.create_dataset("SubrunID",                 data=segHead[:]['subrun'])
outFile.create_dataset("SpillID",                  data=segHead[:]['spill'])
outFile.create_dataset("InteractionID",            data=segHead[:]['interactionID'])
outFile.create_dataset("TrackID",                  data=segHead[:]['trackID'])
outFile.create_dataset("PDG",                      data=segHead[:]['pdg'])
outFile.create_dataset("StartXPos",                data=segPos[:]['start_x'])
outFile.create_dataset("EndXPos",                  data=segPos[:]['end_x'])
outFile.create_dataset("StartYPos",                data=segPos[:]['start_y'])
outFile.create_dataset("EndYPos",                  data=segPos[:]['end_y'])
outFile.create_dataset("StartZPos",                data=segPos[:]['start_z'])
outFile.create_dataset("EndZPos",                  data=segPos[:]['end_z'])
outFile.create_dataset("IonizationElectrons",      data=ionE[:]['nElectrons'])
outFile.create_dataset("LongitudnalDiffusion",     data=ionE[:]['long_diff'])
outFile.create_dataset("TransverseDiffusion",      data=ionE[:]['trans_diff'])

<HDF5 dataset "TransverseDiffusion": shape (1000000,), type "<f8">

In [26]:
inFile.close()
outFile.close()

<font color=black>__The End!__</font>